<img src='https://drive.google.com/uc?export=view&id=1jyAgAg3L5dUSfO9zfp_bXVZrlV8HM-Z9'>




# Import library and Set Working Environment

## Install OpenSilex in the current Environment
OpenSilex GitHub Page referencing Python package installation, and full Documentation for API Endpoints: </Br>
https://github.com/OpenSILEX/opensilexClientToolsPython </Br>
https://opensilex-scripts.pages.mia.inra.fr/opensilex-generic-scripts/scripts/overview

In [ ]:
!pip install git+https://github.com/OpenSILEX/opensilexClientToolsPython.git@1.3.3

## Import all requiered library

In [ ]:
### Import library ##########################################################################################################
import os
import opensilexClientToolsPython as osC
import pandas as pd
import sys
import datetime
import time
from lxml import etree as ET
import json
from tqdm import tqdm
import yaml
from google.colab import data_table

print("Library Imported")

## Setting Up the Working Environment

Clone all data related to this exercise from the NaPPI-specific GitHub repository and Set the working directory.
<img src='https://drive.google.com/uc?export=view&id=1pXvZ1PbqA86Pu09sfsxfnUEsiwxwQdlc'>


In [ ]:
### Clone all Data from GitHub ##############################################################################################
!git clone https://github.com/NaPPI-T/2025_PHIS_Data-Import

### Set Database wd #########################################################################################################
wd = '/content/2025_PHIS_Data-Import'

# Connect to the PHIS Demo instance
http://opensilex.org/sandbox/app/</Br>
Connect as guest


In [ ]:
### Get Login Info ##########################################################################################################
with open (os.path.join(wd, '00-yaml', 'Sandbox-login.yaml'), 'r') as stream:
    login=yaml.safe_load(stream)

### Connect to PHIS Database ################################################################################################
Py_Client = osC.ApiClient()
Py_Client.connect_to_opensilex_ws(identifier=login["Identifier"],
                                  password=login["Password"],
                                  host=login["Host"])
#print(Py_Client.default_headers['Authorization'])
if Py_Client.default_headers:
    print("Connected")

# Get Experiment Info
<img src='https://drive.google.com/uc?export=view&id=1dy_OQTIckBbUxPA1RndMSGR2cM8HVHAa'>


In [ ]:
prefix='00-Sylvain' #CHANGE THIS STRING TO GENERATE YOUR PERSONALIZED EXPERIMENT NAME

### Set Experiment Name #####################################################################################################
NameExp = prefix + "_Potato"
print(f'Experiment Name is "{NameExp}"\n')

### Get Exp Info ############################################################################################################
with open (os.path.join(wd, '00-yaml', 'ExpInfo.yaml'), 'r') as stream:
    ExpInfo=yaml.safe_load(stream)

display(ExpInfo)

# Get or Create Experiment Name&URI

In [ ]:
### Get or Create Scientific Objects Name&URI ###############################################################################
Exp_Api = osC.ExperimentsApi(Py_Client)
Exp_Src = Exp_Api.search_experiments(name=NameExp)["result"]

NameExp_uri = {}
if Exp_Src:
    NameExp_uri.update({NameExp: Exp_Src[0].uri})
    print("Experiment URI:\n{}".format(NameExp_uri[NameExp]))
    del Exp_Api, Exp_Src

    # Get Facilities Name&URI ###############################################################################################
    Org_Api = osC.OrganizationsApi(Py_Client)
    Facilities = ExpInfo['facilities']
    #print("Organization: {}".format(str(Organisation)))
    Facilities_uri = {}
    if Facilities == None:
        print("Organisation Missing")
        ls_Facilities=None
    else:
      for facility in Facilities:
        Org_Src = Org_Api.search_facilities(pattern=facility)["result"]
        if Org_Src:
          Facilities_uri.update({facility: Org_Src[0].uri})
          print("{} URI: {}".format(facility, Org_Src[0].uri))
        else:
            print("\033[91m{}: Unknown Facility\033[0m".format(facility))
            del facility, Org_Src
    del Org_Api, Facilities

else:
    # List Exp Mandatory Info ###############################################################################################
    ObjectiveExp = ExpInfo['Objective']
    if ObjectiveExp != None:
      print("Objective: {}".format(ObjectiveExp))
    else:
      sys.exit("Objective Missing")

    StartExp = ExpInfo['Start Date']
    #print(ExpInfo['Start Date'])
    if StartExp != None:
      print("Start Date: {}".format(StartExp))
    else:
      sys.exit("Starting Date Missing")
    print('#'*50)

    # Get Description Name&URI ##############################################################################################
    DescriptionExp = ExpInfo['Description']
    print("Description: {}".format(str(DescriptionExp)))

    # Get End Date Name&URI #################################################################################################
    EndExp = ExpInfo['End Date']
    #print("End Date: {}".format(EndExp))
    if EndExp != None:
      print("Start Date: {}".format(EndExp))
    else:
      print("Ending Date Missing")

    # Get Is_Public Info ####################################################################################################
    Is_Public = ExpInfo['Is_Public']
    print("Is_Public: {}".format(str(Is_Public)))
    print('#'*50)

    # Get Organisation Name&URI #############################################################################################
    Org_Api = osC.OrganizationsApi(Py_Client)
    Organisation = ExpInfo['organisations']
    #print("Organization: {}".format(str(Organisation)))
    Organisation_uri = {}
    if Organisation == None:
        print("Organisation Missing")
        ls_Organisation=None
    else:
      for organisation in Organisation:
        Org_Src = Org_Api.search_organizations(pattern=organisation)["result"]
        if Org_Src:
          Organisation_uri.update({organisation: Org_Src[0].uri})
          print("{} URI: {}".format(organisation, Org_Src[0].uri))
          ls_Organisation=list(Organisation_uri.values())
        else:
            print("\033[91m{}: Unknown Organisation\033[0m".format(organisation))
            ls_Organisation=None
        del organisation, Org_Src
    del Org_Api, Organisation
    print('#'*50)

    # Get Groups Name&URI ###################################################################################################
    Sec_Api = osC.SecurityApi(Py_Client)
    Groups = ExpInfo['groups']
    #print("Groups: {}".format(str(Groups)))
    Groups_uri = {}
    if Groups == None:
        print("Group Missing")
        ls_Groups=None
    else:
      for group in Groups:
        Sec_Src = Sec_Api.search_groups(name=group)["result"]
        if Sec_Src:
          Groups_uri.update({group: Sec_Src[0].uri})
          print("{} URI: {}".format(group, Sec_Src[0].uri))
          ls_Groups=list(Groups_uri.values())
        else:
            print("\033[91m{}: Unknown Group\033[0m".format(group))
            ls_Groups=None
        del group, Sec_Src
    del Sec_Api, Groups
    print('#'*50)

    # Get Project Name&URI ##################################################################################################
    Proj_Api = osC.ProjectsApi(Py_Client)
    Projects = ExpInfo['projects']
    #print("Projects: {}".format(str(Projects)))
    Projects_uri = {}
    if Projects == None:
        print("Project Missing")
        ls_Projects=None
    else:
        for project in Projects:
            Proj_Src = Proj_Api.search_projects(name=project)["result"]
            if Proj_Src:
                Projects_uri.update({project: Proj_Src[0].uri})
                print("{} URI: {}".format(project, Proj_Src[0].uri))
                ls_Projects=list(Projects_uri.values())
            else:
                print("\033[91m{}: Unknown Project\033[0m".format(project))
                ls_Projects=None
            del project, Proj_Src
    del Proj_Api, Projects
    print('#'*50)

    # Get Facilities Name&URI ###############################################################################################
    Org_Api = osC.OrganizationsApi(Py_Client)
    Facilities = ExpInfo['facilities']
    #print("Organization: {}".format(str(Organisation)))
    Facilities_uri = {}
    if Facilities == None:
        print("Organisation Missing")
        ls_Facilities=None
    else:
      for facility in Facilities:
        Org_Src = Org_Api.search_facilities(pattern=facility)["result"]
        if Org_Src:
          Facilities_uri.update({facility: Org_Src[0].uri})
          print("{} URI: {}".format(facility, Org_Src[0].uri))
          ls_Facilities=list(Facilities_uri.values())
        else:
            print("\033[91m{}: Unknown Facility\033[0m".format(facility))
            ls_Facilities=None
        del facility, Org_Src
    del Org_Api, Facilities
    print('#'*50)

    # Get Scientific Supervisor Name&URI ####################################################################################
    Sec_Api = osC.SecurityApi(Py_Client)
    Scientific_Supervisors = ExpInfo['scientific_supervisors']
    #print("Scientific Supervisors: {}".format(str(Scientific_Supervisors)))
    Scientific_Supervisors_uri = {}
    if Scientific_Supervisors == None:
        print("Scientific Supervisors Missing")
        ls_Scientific_Supervisors=None
    else:
      for scisup in Scientific_Supervisors:
        Sec_Src = Sec_Api.search_persons(name=scisup)["result"]
        if Sec_Src:
          Scientific_Supervisors_uri.update({scisup: Sec_Src[0].uri})
          print("{} URI: {}".format(scisup, Sec_Src[0].uri))
          ls_Scientific_Supervisors=list(Scientific_Supervisors_uri.values())
        else:
            print("\033[91m{}: Unknown Scientific Supervisors\033[0m".format(scisup))
            ls_Scientific_Supervisors=None
        del scisup, Sec_Src
    del Sec_Api, Scientific_Supervisors
    print('#'*50)

    # Get Technical Supervisor Name&URI #####################################################################################
    Sec_Api = osC.SecurityApi(Py_Client)
    Technical_Supervisors = ExpInfo['technical_supervisors']
    #print("Technical Supervisors: {}".format(str(Technical_Supervisors)))
    Technical_Supervisors_uri = {}
    if Technical_Supervisors == None:
        print("Technical Supervisors Missing")
        ls_Technical_Supervisors=None

    else:
      for techsup in Technical_Supervisors:
        Sec_Src = Sec_Api.search_persons(name=techsup)["result"] ### need to search person
        if Sec_Src:
          Technical_Supervisors_uri.update({techsup: Sec_Src[0].uri})
          print("{} URI: {}".format(techsup, Sec_Src[0].uri))
          ls_Technical_Supervisors=list(Technical_Supervisors_uri.values())

        else:
            print("\033[91m{}: Unknown Technical Supervisors\033[0m".format(techsup))
            ls_Technical_Supervisors=None
        del techsup, Sec_Src
    del Sec_Api, Technical_Supervisors
    print('#'*50)

    # Create Experiment #####################################################################################################
    body = osC.ExperimentCreationDTO(
        name=NameExp,
        start_date=StartExp,
        end_date=EndExp,
        description=DescriptionExp,
        objective=ObjectiveExp,
        organisations=ls_Organisation,
        projects=ls_Projects,
        facilities=ls_Facilities,
        scientific_supervisors=ls_Scientific_Supervisors,
        technical_supervisors=ls_Technical_Supervisors,
        groups=ls_Groups,
        is_public=Is_Public)
    Api_Resp = Exp_Api.create_experiment(body=body,)
    print("Experiment Creation: {}".format(str(Api_Resp["metadata"]["datafiles"])))

    ### Get Experiment Name&URI #############################################################################################
    Exp_Src = Exp_Api.search_experiments(name=NameExp)
    NameExp_uri.update({NameExp: Exp_Src["result"][0].uri})
    print( "{} URI: {}".format(NameExp,  NameExp_uri[NameExp]))
    del Exp_Api, body, Api_Resp, DescriptionExp, ObjectiveExp, StartExp, EndExp, Is_Public, Organisation_uri, Groups_uri, Scientific_Supervisors_uri, Technical_Supervisors_uri

# Get RGB1 Numerical Data

In [ ]:
### Define the target datetime format with timezone offset ##################################################################
desired_format = "%Y-%m-%dT%H:%M:%S%z"

df_data = pd.read_excel(os.path.join(wd,'2022_PotatoExp01.xlsx'))
df_data['Tray ID']= prefix + '_' + df_data['Tray ID']
df_data['Measuring Date'] = df_data['Measuring Date'].dt.date
df_data['Measuring Time'] = df_data['Measuring Time'].dt.tz_localize('UTC').dt.tz_convert('Europe/Helsinki').dt.strftime(desired_format)
df_data.head()

# Reference Scientific Object
Your observed object (e.g., Plant, Leaf, Plot) is linked to several metadata elements that provide essential context and description:



*   Has Germplasm: Specifies the genetic material or cultivar used in the
experiment.
*   Has Factor Level: Describes the specific treatment or environmental condition applied (e.g., drought level, fertilizer type, temperature).
*   Has Facility: Indicates the location or controlled environment where the observation was conducted.
*   ...

These metadata help ensure that the scientific object is fully described.

<img src='https://drive.google.com/uc?export=view&id=1Kp79lO6dFdZBorh_Qy1tnjsUBug4g76c'>

## Get or Create Germplasms Name&URI

In [ ]:
### Get Germplasms Info #####################################################################################################
with open (os.path.join(wd, '00-yaml', 'Germplasm.yaml'), 'r') as stream:
    Germplasms=yaml.safe_load(stream)

display(Germplasms)

In [ ]:
### Get Subtaxa Name&URI ####################################################################################################
Germ_Api = osC.GermplasmApi(Py_Client)

Species_uri={}
Germplasms_uri={}

for entry in Germplasms:
    for species, rdftype_sp in entry['Species'].items():

        Germ_Src = Germ_Api.search_germplasm(name=f"^{species}$", # Should search for exact match
                                             rdf_type=rdftype_sp)["result"]

        if Germ_Src:
            Species_uri.update({species: Germ_Src[0].uri})
            print('#'*50)
            print(f"{species}, {rdftype_sp} URI: {Species_uri[species]}")
            del Germ_Src, rdftype_sp

        else:
            check_only = False
            body = osC.GermplasmCreationDTO(
                name=species,
                rdf_type=rdftype_sp)

            Germ_Api.create_germplasm(body=body, check_only=check_only)

            Germ_Src = Germ_Api.search_germplasm(name=f"^{species}$", # Should search for exact match
                                                 rdf_type=rdftype_sp)["result"]

            Species_uri.update({species: Germ_Src[0].uri})
            print('#'*50)
            print(f"{species}, {rdftype_sp} Creation: {Germ_Src[0].uri}")
            del Germ_Src, rdftype_sp, body

        for germplasms in entry['Germplasm']:

            for germplasm, rdftype_g in germplasms.items():

                Germ_Src = Germ_Api.search_germplasm(name=f"^{germplasm}$", # Should search for exact match
                                                     rdf_type=rdftype_g)["result"]

                if Germ_Src:
                    Germplasms_uri.update({germplasm: Germ_Src[0].uri})
                    print(f"{germplasm}, {rdftype_g} URI: {Germplasms_uri[germplasm]}")
                    del Germ_Src, rdftype_g

                else:
                    check_only = False
                    body = osC.GermplasmCreationDTO(
                        name=germplasm,
                        rdf_type=rdftype_g,
                        species=Species_uri[species])

                    Germ_Api.create_germplasm(body=body, check_only=check_only)

                    Germ_Src = Germ_Api.search_germplasm(name=f"^{germplasm}$", # Should search for exact match
                                                         rdf_type=rdftype_g)["result"]

                    Germplasms_uri.update({germplasm: Germ_Src[0].uri})
                    print(f"{germplasm}, {rdftype_g} Creation: {Germ_Src[0].uri}")
                    del Germ_Src, rdftype_g, body
del species, germplasm, Germ_Api

## Get or Create Factors&Levels Names&URI
<img src='https://drive.google.com/uc?export=view&id=1aTTyerKrVJJr_7fNUa6Hu6WCGw5eDy4m'>



In [ ]:
### Get Exp Info ############################################################################################################
with open (os.path.join(wd, '00-yaml', 'Factors.yaml'), 'r') as stream:
    Factors=yaml.safe_load(stream)

display(Factors)

In [ ]:
### Search Factors ##########################################################################################################
Fac_Api = osC.FactorsApi(Py_Client)

Factors_uri = {}
Factors_Levels_uri = {}

for factor in Factors:
    Fac_Src = Fac_Api.search_factors(name=factor,
                                     experiment=NameExp_uri[NameExp])["result"]

    # Get Existing Factors Name&URI #########################################################################################
    if Fac_Src:
        Factors_uri.update({factor: Fac_Src[0].uri})
        del Fac_Src

    # Create Factors ########################################################################################################
    else:
        # Creation of LevelsDTO #############################################################################################
        lvls = {}
        for i in Factors:
            lvl = []
            for j in Factors[i]['Levels']:
                lvl.append(osC.FactorLevelCreationDTO(name=j))
            lvls[i] = lvl

        # Creation of FactorDTO #############################################################################################
        bodies = []
        for i in lvls:
            body = osC.FactorCreationDTO(name=i,
                                         levels=lvls[i],
                                         experiment=NameExp_uri[NameExp],
                                         description=Factors[i]['Description'])
            bodies.append(body)

        # Creation of Factors ###############################################################################################
        for i in bodies:
            Api_Resp = Fac_Api.create_factor(body=i,)
            print("Factors Creation: {}".format(str(Api_Resp["metadata"]["datafiles"])))
        del lvls, lvl, bodies, body, Api_Resp, Fac_Src, i, j

        # Get New Factors Name&URI ##########################################################################################
        Fac_Src = Fac_Api.search_factors(name=factor,
                                         experiment=NameExp_uri[NameExp])["result"]
        Factors_uri.update({factor: Fac_Src[0].uri})

### Get Factors Levels Name&URI #############################################################################################
for fac_uri in Factors_uri.values():
    Fac_Get = Fac_Api.get_factor_levels(uri=fac_uri)["result"]
    for lvl in Fac_Get:
        Factors_Levels_uri.update({lvl.name: lvl.uri})
del factor, fac_uri, Fac_Get, lvl

### Print Factors Name&URI ##################################################################################################
for factor in Factors_uri:
    print("{} URI: {}".format(factor, Factors_uri[factor]))
for lvl in Factors_Levels_uri:
    print("{} URI: {}".format(lvl, Factors_Levels_uri[lvl]))
del factor, lvl, Fac_Api, Factors

## Extract Metadata

In [ ]:
### Get PID from DataFrame ##################################################################################################
PID = df_data['PID'].unique()[0]
print(f'PID found: {PID}')

### Create a new dataframe with unique rows from 'Tray ID' to 'Factor Level', dropping duplicates and NaN values ############
df_ScObj = df_data.loc[:, "Tray ID":"Factor Level"].drop_duplicates().dropna()

# Display the first few rows of the new dataframe
df_ScObj.head()

## Get or Create Scientific Objects Name&URI
<img src='https://drive.google.com/uc?export=view&id=1Nb9NzE8TfUAbIwHifj-fwtIWbmHVfwho'>

<img src='https://drive.google.com/uc?export=view&id=1uhCIhLu9mu96e8FXhJWzTdonbV5eUW9B'>


In [ ]:
### Get or Create Scientific Objects Name&URI ###############################################################################
Relations_Gen = []

### ObjectRealtionDTO for Start Date ########################################################################################
StartExp = ExpInfo['Start Date']
if StartExp != None:
    relation_temp = osC.RDFObjectRelationDTO(
        _property="vocabulary:hasCreationDate",
        value=StartExp)
    Relations_Gen.append(relation_temp)
    del relation_temp, StartExp

else:
    print('Start Date Missing')

EndExp = ExpInfo['End Date']
if EndExp != None:
    relation_temp = osC.RDFObjectRelationDTO(
        _property="vocabulary:hasDestructionDate",
        value=EndExp)
    Relations_Gen.append(relation_temp)
    del relation_temp, EndExp
else:
    print('End Date Missing')

### Scientific Object RDF Type ##############################################################################################
BioMat_Type = ExpInfo['RDF Type']
if not BioMat_Type:
    sys.exit("Scientific Object RDF Type Missing")
else:
    for biomat in BioMat_Type:
        Onto_Api = osC.OntologyApi(Py_Client)
        Onto_Src = Onto_Api.search_sub_classes_of(
            name=biomat, parent_type="vocabulary:ScientificObject")["result"]
        if Onto_Src:
            rdf_type = Onto_Src[0].children[0].uri
        else:
            sys.exit("Scientific Object RDF Type Unknown")
    del biomat, BioMat_Type, Onto_Api, Onto_Src

### Get Scientific Objects Name&URI #########################################################################################
ScObj_Api = osC.ScientificObjectsApi(Py_Client)
ScObj_uri = {}
for index, row in tqdm(df_ScObj.iterrows(), desc="ScObj processing:"):
    ScObj_Src = ScObj_Api.search_scientific_objects(name=row["Tray ID"])["result"]
    if ScObj_Src:
        ScObj_uri.update({row["Tray ID"]: ScObj_Src[0].uri})
    else:
        Relations_ScObj=[]

        # ObjectRealtionDTO for Germplasm ###################################################################################
        if Germplasms_uri:
            relation_temp = osC.RDFObjectRelationDTO(
                _property="vocabulary:hasGermplasm",
                value=Germplasms_uri.get(row["Germplasm"]))
            Relations_ScObj.append(relation_temp)
            del relation_temp

        # ObjectRealtionDTO for Factors #####################################################################################
        if Factors_Levels_uri:
            relation_temp = osC.RDFObjectRelationDTO(
                _property="vocabulary:hasFactorLevel",
                value=Factors_Levels_uri.get(row["Factor Level"]))
            Relations_ScObj.append(relation_temp)

        Relations = Relations_Gen + Relations_ScObj

        # Creation of Scientific Object #####################################################################################
        body = osC.ScientificObjectCreationDTO(name=row["Tray ID"],
                                               rdf_type=rdf_type,
                                               relations=Relations,
                                               experiment=NameExp_uri[NameExp])
        ScObj_Api.create_scientific_object(body, )
        del Relations, Relations_ScObj

        # Get New Scientific Object Name&URI ################################################################################
        ScObj_Src = ScObj_Api.search_scientific_objects(name=row["Tray ID"])["result"]
        ScObj_uri.update({row["Tray ID"]: ScObj_Src[0].uri})
print("Done")
del index, row, ScObj_Src, ScObj_Api, Relations_Gen, rdf_type

display(ScObj_uri)

# Get Metadata related to Imaging
<img src='https://drive.google.com/uc?export=view&id=1IDKxnr5vowpPx8cvoLeCcTOaFDl4SCZW'>

In our image processing pipeline, we use specific software that allows us to remove image backgrounds (e.g., generate masked images) by using a set of parameters, including a color formula, threshold and other settings. These settings, being color-dependent, change from species to species and also depend on the age of the plant.

## Get Round Protocol Info
<img src='https://drive.google.com/uc?export=view&id=1-CkTjThXGncWvmOOzjRPvaKYTu-UBS5L'>

In our NaPPI system, all parameters used by our software to remove image backgrounds (e.g., generate masked images) are stored in RoundProtocol files as XML-like files. This part of the script recovers the masking parameters for every imaging event.

In [ ]:
### Get RoundProtocol Infos #################################################################################################
Round_folder = os.path.join(wd, f'RoundProtocol_{PID}')
Round_files = []
for (root, dirs, files) in os.walk(Round_folder):
    for name in files:
        Round_files.append(os.path.join(root, name))
del root, dirs, files, name

## Link RoundProtocol wd with Experiment and Round Numbers ########################################
Exp_Rd_Dict = {}
for wd_round in Round_files:
    Exp_Rd_temp = os.path.basename(wd_round).replace("RoundProtocol-", "")
    Exp_Rd = Exp_Rd_temp.replace(".txt", "")
    Exp_Rd_ls = Exp_Rd.split("-")
    Exp_Rd_Dict.update({wd_round: {"Experiment": Exp_Rd_ls[0], "Round": Exp_Rd_ls[1]}})
del wd_round, Exp_Rd_temp, Exp_Rd, Exp_Rd_ls, Round_files, Round_folder

## Create Dictionary for all parameters ###########################################################
PlantMask = {}
CamPos = {}
ColorSeg = {}

## Transform RoundProtocol to acceptable xml ######################################################
for key, value in Exp_Rd_Dict.items():
    with open(key) as file:
        xml_like = file.read()
    file.close()
    xml_str = str.replace(xml_like, "\x00", "", -1)
    root = ET.fromstring(xml_str)
    del file, xml_like, xml_str

## Get PlantMask Info for all rounds ##############################################################
    PlantMask_rd = {}
    for child in root.iter(PID):
        for subchild in child.iter("PlantMask"):
            for i in subchild.iter():
                PlantMask_rd.update({i.tag: i.text})
    if 'PlantMask' in PlantMask_rd:
        del PlantMask_rd['PlantMask']
    PlantMask.update({value["Round"]: PlantMask_rd})
    del child, subchild, i, PlantMask_rd
## Get Camera Position Info for all rounds ########################################################
    CamPos_rd = {}
    for child in root.iter(PID):
        CamPos_rd.update(child.attrib)
    for child in root.iter(PID):
        for subchild in child.iter("Offset"):
            CamPos_rd.update({subchild.tag: subchild.text})
    CamPos.update({value["Round"]: CamPos_rd})
    del child, subchild, CamPos_rd

print("PlantMask, and Camera Position Info Listed")
del key, value, Exp_Rd_Dict, root

Display Plant Mask parameters of the round 11

In [ ]:
PlantMask['11']

Display Plant Mask parameters of the round 4

In [ ]:
CamPos['4']

## Get links between Image Names and Timestamp
<img src='https://drive.google.com/uc?export=view&id=1paS-GZk_MGG9UnuwR2P7xN2rmxunMSTs'>

In our system, all image names follow a specific nomenclature referencing the experiment's unique ID, imaging round order, and tray ID. This allows us to match each specific image to a unique data point.

In [ ]:
### Add Image Names to df_data ##############################################################################################
df_data['Img Name'] = df_data.apply(lambda row: f"{row['Experiment ID']}-{row['Round Order']}-{row['Tray ID']}-{row['PID']}-{row['Angle']:03}", axis=1)
display(df_data.head())

### Link Image Names and TimeStamp ##########################################################################################
TimeStamp={}
for index, row in df_data.iterrows():
    TimeStamp.update({row["Img Name"]: row['Measuring Time']})
del index, row

first_key = next(iter(TimeStamp))
first_value = TimeStamp[first_key]
print("Img Name:", first_key)
print("Measuring Time:", first_value)

In [ ]:
### Display link between images name and its timestamp ######################################################################
TimeStamp

# Get Provenance Name&URI
<img src='https://drive.google.com/uc?export=view&id=1w9l9cemRWtvfsus7p7WXHBof4i7yCcN0'>

## Get or Create FishEyeCorrected Images Provenance Name&URI

In [ ]:
### Initialize an empty dictionary to store provenance data and set the facility type #######################################
prov_dict = {}
Facility='Modular'

### Get Fish-Eye Corrected Provenance #######################################################################################
Dat_Api = osC.DataApi(Py_Client)

prov = str("{}_{}_FishEyeCorrectedImages".format(Facility, PID))

Prov_Src = Dat_Api.search_provenance(name=prov,)["result"]
if Prov_Src:
    prov_dict.update({prov: Prov_Src[0].uri})
    print("{} URI: {}".format(prov,Prov_Src[0].uri))
else:
    description = "Fish-Eye Corrected images acquired by Modular line scan camera (RGB1)"

    prov_activity = [osC.ActivityCreationDTO(rdf_type="vocabulary:ImageAcquisition")]

    prov_agent = [
        osC.AgentModel(uri="opensilex-sandbox:id/device/modular_plantscreen",
                           rdf_type="vocabulary:Actuator"),

        osC.AgentModel(uri="opensilex-sandbox:id/device/modular_linescan_rgb1",
                                 rdf_type="vocabulary:SensingDevice",
                                 settings={})]

    body = osC.ProvenanceCreationDTO(name=prov,
                                     description=description,
                                     prov_agent=prov_agent,
                                     prov_activity=prov_activity)

    Api_Resp = Dat_Api.create_provenance(body=body, )
    print("Provenance Created: {}".format(str(Api_Resp["metadata"]["datafiles"])))
    Prov_Src = Dat_Api.search_provenance(name=prov,)["result"]
    prov_dict.update({prov: Prov_Src[0].uri})
    print("{} URI Created: {}".format(prov,  Prov_Src[0].uri))
    del description, prov_activity, prov_agent, body, Api_Resp
del prov, Dat_Api, Prov_Src

## Get or Create FishEyeMasked Images Provenance Name&URI

In [ ]:
### Get Fish-Eye Masked Provenance ##########################################################################################
Dat_Api = osC.DataApi(Py_Client)

prov = str("{}_{}_FishEyeMaskedImages".format(Facility, PID))

Prov_Src = Dat_Api.search_provenance(name=prov,)["result"]
if Prov_Src:
    prov_dict.update({prov: Prov_Src[0].uri})
    print("{} URI: {}".format(prov, Prov_Src[0].uri))
else:
    description = "Fish-Eye Masked images generated by PSI DataAnalyser from Modular line scan (RGB1) images"

    prov_activity = [osC.ActivityCreationDTO(rdf_type="vocabulary:ImageAnalysis")]

    prov_agent = [
        osC.AgentModel(uri="opensilex-sandbox:id/device/plantscreen_data_analyser",
                       rdf_type="vocabulary:Software"),

        osC.AgentModel(uri="opensilex-sandbox:id/device/modular_linescan_rgb1",
                                 rdf_type="vocabulary:SensingDevice",
                                 settings={})
        ]

    body = osC.ProvenanceCreationDTO(name=prov,
                                     description=description,
                                     prov_agent=prov_agent,
                                     prov_activity=prov_activity)

    Api_Resp = Dat_Api.create_provenance(body=body, )
    print("Provenance Created: {}".format(str(Api_Resp["metadata"]["datafiles"])))
    Prov_Src = Dat_Api.search_provenance(name=prov,)["result"]
    prov_dict.update({prov: Prov_Src[0].uri})
    print("{} URI Created: {}".format(prov, Prov_Src[0].uri))
    del description, prov_activity

## Get or Create Morphological Parameters Provenance Name&URI

In [ ]:
### Get MorphoParameters Provenance #########################################################################################
Dat_Api = osC.DataApi(Py_Client)

prov = str("{}_{}_MorphoParameters".format(Facility, PID))

Prov_Src = Dat_Api.search_provenance(name=prov,)["result"]
if Prov_Src:
    prov_dict.update({prov: Prov_Src[0].uri})
    print("{} URI: {}".format(prov, Prov_Src[0].uri))
else:
    description = "Morphological parameters computed by PSI DataAnalyser from Modular line scan (RGB1) images"
    prov_activity = [osC.ActivityCreationDTO(rdf_type="vocabulary:ImageAnalysis")]
    prov_agent = [

        osC.AgentModel(uri="opensilex-sandbox:id/device/modular_plantscreen",
                       rdf_type="vocabulary:Actuator"),

        osC.AgentModel(uri="opensilex-sandbox:id/device/modular_linescan_rgb1",
                       rdf_type="vocabulary:SensingDevice",
                       settings={}),

        osC.AgentModel(uri="opensilex-sandbox:id/device/plantscreen_data_analyser",
                       rdf_type="vocabulary:Software")
        ]

    body = osC.ProvenanceCreationDTO(name=prov,
                                     description=description,
                                     prov_agent=prov_agent,
                                     prov_activity=prov_activity)

    Api_Resp = Dat_Api.create_provenance(body=body, )
    print("Provenance Created: {}".format(str(Api_Resp["metadata"]["datafiles"])))
    Prov_Src = Dat_Api.search_provenance(name=prov,)["result"]
    prov_dict.update({prov: Prov_Src[0].uri})
    print("{} URI Created: {}".format(prov, Prov_Src[0].uri))
    del description, prov_activity, prov_agent, body, Api_Resp
del prov, Dat_Api, Prov_Src

# Import datafiles (Images)
This section handles the import of all image data files.

## List all images

In [ ]:
### Initialize an empty list to store file paths ############################################################################
ls_files = []

### Walk through the directory tree #########################################################################################
for (root, dirs, files) in os.walk(wd):
    # Iterate over each file in the current directory
    for filename in files:
        # Check if the file has a .png extension
        if filename.endswith(".png"):
            # Add the full file path to the list
            ls_files.append(os.path.join(root, filename))

### Create a list of files that contain '-FishEyeCorrected' or '-FishEyeMasked' in their names ##############################
ls_fec = [x for x in ls_files if "-FishEyeCorrected" in x]
ls_fem = [x for x in ls_files if "-FishEyeMasked" in x]

### Print the number of FishEyeCorrected and FishEyeMasked images ###########################################################
print(f'Number of FEC Images: {len(ls_fec)}\nNumber of FEM Images: {len(ls_fem)}')

## FishEyeCorrected Images

### Get Images Metadata
<img src='https://drive.google.com/uc?export=view&id=1izWUouapB9A3MGyvIvukZ_ioHQkbrKPI'>

In [ ]:
### Set the Provenance ######################################################################################################
prov='Modular_RGB1_FishEyeCorrectedImages'

### Initialize an empty list to store all the dictionaries ##################################################################
Corr_Name_All = []

### Loop through each file in ls_fec ########################################################################################
for corr in ls_fec:
    # Extract the base filename and remove the ".png" extension
    Corr_temp1 = os.path.basename(corr).replace(".png", "")
    # Split the filename into its components
    Corr_ls = Corr_temp1.split("-")
    Corr_temp2 = Corr_ls[0]+'-'+Corr_ls[1]+'-'+ prefix + '_' +Corr_ls[2]+'-'+Corr_ls[3]+'-'+Corr_ls[4]

    # Create the dictionary with common keys
    Corr_Dict = {
        "Path": corr,
        "Experiment ID": Corr_ls[0],
        "Round Order": Corr_ls[1],
        "Date": str(TimeStamp[Corr_temp2]),
        "Tray ID": prefix + '_' + Corr_ls[2],
        "PID": Corr_ls[3],
        'Prov': prov_dict[prov],
        "ImgType": Corr_ls[-1]
    }

    # Add the "Angle" key only if PID is 'RGB1'
    if PID == 'RGB1':
        Corr_Dict["Angle"] = Corr_ls[4]
    # Append the dictionary to the list
    Corr_Name_All.append(Corr_Dict)

### Display the first dictionary in the list ################################################################################
display(Corr_Name_All[0])
print(len(Corr_Name_All))

### Check if any Images already exist

In [ ]:
### Filter out existing FEC #################################################################################################
Dat_Api = osC.DataApi(Py_Client)
Dat_Src = Dat_Api.get_data_file_descriptions_by_search(provenances=[prov_dict[prov]], experiments=[NameExp_uri[NameExp]], page_size=100000)["result"]
Corr_Name_Com=[]
for i in Corr_Name_All:
    for elts in Dat_Src:
        if ScObj_uri[i["Tray ID"]] == elts.target and i["Date"].replace('+', '.000+') == elts._date and i["Angle"] == elts.provenance.settings["Camera Angle"] and i["Round Order"] == elts.metadata['Round Order']:
        #if ScObj_uri[i["Tray ID"]] == elts.target and i["Date"].replace('+', '.000+') == elts._date:
            Corr_Name_Com.append(i)
            #print("ScObj {} with angle {} at date {} already  exist".format(ScObj_uri[i["Name"]], i["Angle"], i["Date"]))
        del elts
del i
print("{} images over {} already exit on the database".format(len(Corr_Name_Com), len(Corr_Name_All)))

### Exclude existing one from the Dict ######################################################################################
Corr_Name=[]
for i in Corr_Name_All:
        if i not in Corr_Name_Com:
            Corr_Name.append(i)
#del i, Corr_Name_All, Corr_Name_Com, Dat_Src

### Import Images
<img src='https://drive.google.com/uc?export=view&id=1bYlmOt8GFK0ZGYqR9Ireys_jsj6ks-qk'>

In [ ]:
### Refresh Connection to the Database ######################################################################################
# Connect to the OpenSILEX web service using the provided login credentials
Py_Client.connect_to_opensilex_ws(identifier=login["Identifier"],
                                  password=login["Password"],
                                  host=login["Host"])

# Set a time limit of 30 minutes from the current time
timelimit = datetime.datetime.now() + datetime.timedelta(minutes=30)

### Import FishEyeCorrected Images ##########################################################################################
# Initialize the Data API with the Py_Client instance
Dat_Api = osC.DataApi(Py_Client)

# Iterate over each image in the Corr_Name list
for img in tqdm(Corr_Name):
  # Create a description dictionary for the image
  description = {
      "rdf_type": "vocabulary:RGBImage",  # Originally proposed as a DTO object
      "date": img["Date"],
      "target": ScObj_uri[img["Tray ID"]],
      "metadata": {"Round Order": img["Round Order"]},
      "provenance": {
          "uri": img["Prov"],
          "settings": {
              "Camera Angle": img["Angle"],
              "Camera Height": CamPos[img["Round Order"]]["height"],
              "Offset": CamPos[img["Round Order"]]["Offset"]
          },
          "experiments": [NameExp_uri[NameExp]]
      }
  }
  # Post the image file along with its description to the Data API
  Dat_Api.post_data_file(description=json.dumps(description), file=img["Path"])  # DTO object replaced by json.dumps()
  # Clean up by deleting the image and description variables
  del img, description

  # Reconnect to the Database After 30 Minutes ##############################################################################
  # Check if the current time exceeds the time limit
  if datetime.datetime.now() > timelimit:
      # Reconnect to the OpenSILEX web service using the provided login credentials
      Py_Client.connect_to_opensilex_ws(identifier=login["Identifier"],
                                        password=login["Password"],
                                        host=login["Host"])
      # Reinitialize the Data API with the Py_Client instance
      Dat_Api = osC.DataApi(Py_Client)
      # Reset the time limit to 30 minutes from the current time
      timelimit = datetime.datetime.now() + datetime.timedelta(minutes=30)
      # print(Py_Client.default_headers['Authorization'])

print('Done')

### Get Created Datafile URI

In [ ]:
### List FEC URI ############################################################################################################
Dat_Api = osC.DataApi(Py_Client)
Dat_Src = Dat_Api.get_data_file_descriptions_by_search(provenances=[prov_dict[prov]], experiments=[NameExp_uri[NameExp]], page_size=100000)["result"]
Corr_uri=[]
for elts in Dat_Src:
    for k, v in ScObj_uri.items():
        if v == elts.target:
            trayid=k

    Corr_uri.append({'Type': 'FEC',
                     "Target": [value for key, value in ScObj_uri.items() if key == trayid][0],
                     "Tray ID": trayid,
                     "Date": elts._date,
                     'Round Order': elts.metadata["Round Order"],
                     "Angle": elts.provenance.settings["Camera Angle"],
                     "uri": elts.uri})
#del elts, k, v, Dat_Src, Dat_Api

display(Corr_uri[0])

## FishEyeMasked Images

### Get Images Metadata
<img src='https://drive.google.com/uc?export=view&id=10RDHKK0pyPB4PZpf7Dkdqz-s4xFU1G9q'>



In [ ]:
### Set the Provenance ######################################################################################################
prov = 'Modular_RGB1_FishEyeMaskedImages'

### Initialize an Empty List to Store All the Dictionaries ##################################################################
Mask_Name_All = []

### Iterate over each file in the list of FishEyeMasked images ##############################################################
for mask in ls_fem:
    # Extract the base name of the file and remove the .png extension
    Mask_temp1 = os.path.basename(mask).replace(".png", "")
    # Split the base name into components
    Mask_ls = Mask_temp1.split("-")
    # Construct a temporary identifier using the components and a prefix
    Mask_temp2 = Mask_ls[0] + '-' + Mask_ls[1] + '-' + prefix + '_' + Mask_ls[2] + '-' + Mask_ls[3] + '-' + Mask_ls[4]

    # Create a dictionary with metadata for the current image
    Mask_Dict = {
        "Path": mask,
        "Experiment ID": Mask_ls[0],
        "Round Order": Mask_ls[1],
        "Date": str(TimeStamp[Mask_temp2]),
        "Tray ID": prefix + '_' + Mask_ls[2],
        "PID": Mask_ls[3],
        'Prov': prov_dict[prov],
        "ImgType": Mask_ls[-1]
    }

    # Add the "Angle" key only if the PID is 'RGB1'
    if PID == 'RGB1':
        Mask_Dict["Angle"] = Mask_ls[4]

    # Append the dictionary to the list
    Mask_Name_All.append(Mask_Dict)

# Clean up temporary variables
del Mask_temp1, Mask_temp2, Mask_ls, mask, Mask_Dict

### Update each dictionary in the list with provenance information ##########################################################
for elts in Mask_Name_All:
    for item in Corr_uri:
        # Check if the Tray ID and Date match, and update the dictionary with the provenance URI
        #if item["Tray ID"]==elts["Tray ID"] and item["Date"]==elts["Date"].replace('+', '.000+') and item["Angle"]==elts["Angle"]:
        if item["Tray ID"] == elts["Tray ID"] and item["Date"] == elts["Date"].replace('+', '.000+'):
            elts.update({"Prov_Used": item["uri"]})

# Display the first dictionary in the list
display(Mask_Name_All[0])

# Print the total number of dictionaries in the list
len(Mask_Name_All)

### Check if any Images already exist

In [ ]:
### Filter out existing FEM #################################################################################################
Dat_Api = osC.DataApi(Py_Client)
Dat_Src = Dat_Api.get_data_file_descriptions_by_search(provenances=[prov_dict[prov]], experiments=[NameExp_uri[NameExp]], page_size=100000)["result"]
Mask_Name_Com=[]
for i in Mask_Name_All:
    for elts in Dat_Src:
        if ScObj_uri[i["Tray ID"]] == elts.target and i["Date"].replace('+', '.000+') == elts._date and i["Angle"] == elts.provenance.settings["Camera Angle"] and i["Round Order"] == elts.metadata['Round Order']:
        #if ScObj_uri[i["Tray ID"]] == elts.target and i["Date"].replace('+', '.000+') == elts._date:
            Mask_Name_Com.append(i)
            #print("ScObj {} with angle {} at date {} already  exist".format(ScObj_uri[i["Name"]], i["Angle"], i["Date"]))
print("{} images over {} already exit on the database".format(len(Mask_Name_Com), len(Mask_Name_All)))
del i

### Exclude existing one from the Dict ######################################################################################
Mask_Name=[]
for i in Mask_Name_All:
        if i not in Mask_Name_Com:
            Mask_Name.append(i)
del i, Mask_Name_All, Mask_Name_Com, Dat_Src

### Import Images
<img src='https://drive.google.com/uc?export=view&id=12VhPtbk2dBjSfzF6LEUaqFs4daig5yRS'>

In [ ]:
### Refresh Connection to the Database ######################################################################################
# Connect to the OpenSILEX web service using the provided login credentials
Py_Client.connect_to_opensilex_ws(identifier=login["Identifier"],
                                  password=login["Password"],
                                  host=login["Host"])

# Set a time limit of 30 minutes from the current time
timelimit = datetime.datetime.now() + datetime.timedelta(minutes=30)

### Import FishEyeMasked Images #############################################################################################
# Initialize the Data API with the Py_Client instance
Dat_Api = osC.DataApi(Py_Client)

# Iterate over each image in the Mask_Name list
for img in tqdm(Mask_Name):
    # Create a dictionary with camera settings for the current image
    Setting_Dict = {"Camera Angle": img["Angle"]}
    Setting_Dict.update(PlantMask[str(img["Round Order"])])

    # Create a description dictionary for the image
    description = {
        "rdf_type": "vocabulary:RGBImage",
        "date": img["Date"],
        "target": ScObj_uri[img["Tray ID"]],
        "metadata": {"Round Order": img["Round Order"]},
        "provenance": {
            "uri": img["Prov"],
            "prov_used": [{"uri": img["Prov_Used"], "rdf_type": "vocabulary:RGBImage"}],
            "settings": Setting_Dict,
            "experiments": [NameExp_uri[NameExp]]
        }
    }

    # Post the image file along with its description to the Data API
    Dat_Api.post_data_file(description=json.dumps(description), file=img["Path"])
    # Clean up by deleting the image and description variables
    del img, description

    # Reconnect to the Database After 30 Minutes ############################################################################
    # Check if the current time exceeds the time limit
    if datetime.datetime.now() > timelimit:
        # Reconnect to the OpenSILEX web service using the provided login credentials
        Py_Client.connect_to_opensilex_ws(identifier=login["Identifier"],
                                          password=login["Password"],
                                          host=login["Host"])
        # Reinitialize the Data API with the Py_Client instance
        Dat_Api = osC.DataApi(Py_Client)
        # Reset the time limit to 30 minutes from the current time
        timelimit = datetime.datetime.now() + datetime.timedelta(minutes=30)
        # Uncomment the following line to print the authorization header
        # print(Py_Client.default_headers['Authorization'])

print('Done')

### Get Created Datafile URI

In [ ]:
### List FEM URI ############################################################################################################
Dat_Api = osC.DataApi(Py_Client)
Dat_Src = Dat_Api.get_data_file_descriptions_by_search(provenances=[prov_dict[prov]], experiments=[NameExp_uri[NameExp]], page_size=100000)["result"]
Mask_uri=[]
for elts in Dat_Src:
    for k, v in ScObj_uri.items():
        if v == elts.target:
            trayid=k

    Mask_uri.append({'Type': 'FEM',
                     "Target": [value for key, value in ScObj_uri.items() if key == trayid][0],
                     "Tray ID": trayid,
                     "Date": elts._date,
                     'Round Order': elts.metadata["Round Order"],
                     "Angle": elts.provenance.settings["Camera Angle"],
                     "uri": elts.uri})
del elts, k, v, Dat_Src, Dat_Api

display(Mask_uri[0])
len(Mask_uri)

# Import Numerical Data
<img src='https://drive.google.com/uc?export=view&id=1w9l9cemRWtvfsus7p7WXHBof4i7yCcN0'>

# Import Data

## Get link between Columns Names and Variables
This section establishes the mapping between observed variables and their corresponding column names in the dataset.

In [ ]:
### Construct the path to the '00-yaml' directory within the working directory ##############################################
wd_yaml = os.path.join(wd, '00-yaml')

### Open the 'Morpho_Info.yaml' file in read mode and load its contents #####################################################
with open(os.path.join(wd_yaml, 'Morpho_Info.yaml'), 'r') as stream:
    Morpho_Info = yaml.safe_load(stream)
del(wd_yaml)

### Display the contents of the 'Morpho_Info' dictionary ####################################################################
display(Morpho_Info)

In [ ]:
display(df_data.head())
print(len(df_data))

## PHIS data import
This section handles the import of numerical data from the dataset.

In [ ]:
### Set the Provenance ######################################################################################################
prov = 'Modular_RGB1_MorphoParameters'

### Connect to the OpenSILEX web service using the provided login credentials ###############################################
Py_Client.connect_to_opensilex_ws(identifier=login["Identifier"],
                                  password=login["Password"],
                                  host=login["Host"])

# Set a time limit of 30 minutes from the current time
timelimit = datetime.datetime.now() + datetime.timedelta(minutes=30)

### Data Import #############################################################################################################
# Initialize the Data API and Variables API with the Py_Client instance
Dat_Api = osC.DataApi(Py_Client)
Var_Api = osC.VariablesApi(Py_Client)

# Define the provenance entity model
Prov_Was_Associated_With = osC.ProvEntityModel(uri="opensilex-sandbox:id/device/plantscreen_data_analyser",
                                               rdf_type="vocabulary:Software")

# Initialize an empty dictionary to log duplicate data
logfile = {}

### Iterate over each item in the Morpho_Info dictionary ####################################################################
for key, value in tqdm(Morpho_Info.items()):
    logfile[value] = []
    # Search for the variable source using the variable name
    Var_Src = Var_Api.search_variables(name=value)["result"]

    # Get or Create Numerical Data ##########################################################################################
    pas = 1000  # Set the batch size for data import
    count = 0  # Initialize a counter

    # Iterate over the data in slices of size 'pas'
    for slc in range(0, len(df_data), pas):
        df_Slice = df_data.iloc[slc:slc + pas]  # Slice the dataframe
        bodies = []  # Initialize an empty list to store data bodies
        count += 1  # Increment the counter

        # Iterate over each row in the sliced dataframe
        for index, row in df_Slice.iterrows():
            # Search for existing data in the Data API
            Dat_Src = Dat_Api.search_data_list(targets=[ScObj_uri[row["Tray ID"]]],
                                               metadata=json.dumps({'Round Order': row['Round Order']}),
                                               start_date=row['Measuring Time'].replace('+', '.000+'),
                                               end_date=row['Measuring Time'].replace('+', '.000+'),
                                               variables=[Var_Src[0].uri],
                                               experiments=[NameExp_uri[NameExp]], page_size=20)['result']
            if Dat_Src:
                # Log the data if it already exists
                logfile[value].append({'Angle': {"{:03d}".format(row["Angle"])}, 'Tray ID': {row["Tray ID"]}, 'Round Order': {row["Round Order"]}})
            else:
                Prov_Used = None  # Initialize the provenance used variable
                Setting_Dict = {"Camera Angle": "{:03d}".format(row["Angle"])}  # Create a settings dictionary

                # Search for the corresponding mask URI
                for item in Mask_uri:
                    if item["Tray ID"] == row["Tray ID"] and item["Date"] == row['Measuring Time'].replace('+', '.000+'):
                        Prov_Used = osC.ProvEntityModel(uri=item["uri"], rdf_type="vocabulary:RGBImage")

                # Create a data creation DTO object
                body = osC.DataCreationDTO(_date=str(row['Measuring Time']),
                                           target=ScObj_uri[row["Tray ID"]],
                                           variable=Var_Src[0].uri,
                                           value=row[key],
                                           metadata={"Round Order": row["Round Order"]},
                                           provenance=osC.DataProvenanceModel(
                                               uri=prov_dict[prov],
                                               prov_used=[Prov_Used],
                                               prov_was_associated_with=[Prov_Was_Associated_With],
                                               settings=Setting_Dict,
                                               experiments=[NameExp_uri[NameExp]]))
                bodies.append(body)  # Append the DTO object to the list

                # Reconnect to the OpenSILEX web service if the time limit is exceeded
                if datetime.datetime.now() > timelimit:
                    Py_Client.connect_to_opensilex_ws(identifier=login["Identifier"],
                                                      password=login["Password"],
                                                      host=login["Host"])
                    Dat_Api = osC.DataApi(Py_Client)
                    timelimit = datetime.datetime.now() + datetime.timedelta(minutes=30)

        if bodies:
            # Add the list of data bodies to the Data API
            Dat_Api.add_list_data(body=bodies)
        else:
            print(f'all data of {value} already uploaded')

print('Import Over')